# 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import pickle

from freeman.task.trading.config import *

# 전처리 데이터 불러오기

In [2]:
train_df = pd.read_csv("/home/freeman/projects/data/trading/data/train_5930.csv", sep=",")
test_df = pd.read_csv("/home/freeman/projects/data/trading/data/test_5930.csv", sep=",")

In [3]:
with open(f"/home/freeman/projects/data/trading/temp/max_value.pkl", "rb") as f:
    max_value = pickle.load(f)

In [4]:
max_value["high"]

96800.0

In [5]:
train_x, train_y = train_df.iloc[:, :-1].copy(deep=True), train_df.iloc[:, -1:]
test_x, test_y = test_df.iloc[:, :-1].copy(deep=True), test_df.iloc[:, -1:]

In [6]:
train_x.columns

Index(['date', 'open', 'low', 'high', 'close', 'diff', 'ratio', 'volume',
       'value', 't_value', 't_volume', 'year', 'day_of_week'],
      dtype='object')

In [7]:
train_x.iloc[:, 0]

0       2018-05-04
1       2018-05-08
2       2018-05-09
3       2018-05-10
4       2018-05-11
           ...    
1008    2022-06-10
1009    2022-06-13
1010    2022-06-14
1011    2022-06-15
1012    2022-06-16
Name: date, Length: 1013, dtype: object

In [8]:
train_x = train_x.iloc[:, 1:]

In [9]:
from sklearn.tree import DecisionTreeRegressor

In [10]:
dtr = DecisionTreeRegressor(max_depth=5)
dtr.fit(train_x, train_y)

DecisionTreeRegressor(max_depth=5)

In [11]:
dtr_pred_y = dtr.predict(test_x.iloc[:, 1:])
test_yy = test_y.to_numpy()
test_yy = np.squeeze(test_yy, axis=1)
test_yy.shape

(79,)

In [12]:
from sklearn.metrics import mean_squared_error
dtr_mse = mean_squared_error(dtr_pred_y, test_yy)

In [13]:
dtr_mse, dtr_mse*max_value["high"]

(0.004818246594561378, 466.40627035354146)

In [14]:
from sklearn.ensemble import ExtraTreesRegressor

In [15]:
etr = ExtraTreesRegressor()
etr.fit(train_x, train_y)

/tmp/ipykernel_31302/158887813.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  etr.fit(train_x, train_y)


ExtraTreesRegressor()

In [16]:
etr_pred_y = etr.predict(test_x.iloc[:, 1:])

In [17]:
etr_mse = mean_squared_error(etr_pred_y, test_yy)
etr_mse, etr_mse*max_value["high"]

(0.005083387549777849, 492.07191481849577)

In [18]:
from pycaret.regression import *

In [20]:
setup_rgs = setup(data=train_x.to_numpy(), target=train_y, session_id=10)
top5 = compare_models(sort="MSE", n_select=5)

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
omp = create_model('omp')

In [ ]:
tuned_omp = tune_model(omp)

In [ ]:
tuned = [tuned_et, tuned_cat, tuned_ada, tuned_rf, tuned_omp, tuned_gbr]
blender = blend_models(tuned)

In [ ]:
pred_y = predict_model(blender_top5, data=test_x)
pred_y